# Performance

Obviously, using unit-aware variables will slow down any computation compared to raw python values (int, flot, numpy.ndarray).

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import physipy
from physipy import s, m, setup_matplotlib

from physipy import Dimension, units, quantify, Quantity



ms = units["ms"]
mm = units['mm']
km = units["km"]
cm = units["cm"]
mus = units["mus"]
ns = units["ns"]
a = 123456
b = 654321

aq = a*m
bq = b*m


Basic comparison on addition

In [2]:
%timeit  (a +  b)
%timeit (aq + bq)

61.8 ns ± 1.8 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
11.8 µs ± 40.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [3]:
print(12.4*mus/(63.7*ns))

194.66248037676607


Basic comparison on pow

In [4]:
%timeit  (a**2)
%timeit (aq**2)

302 ns ± 9.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
21.2 µs ± 2.04 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
print(22.8*mus/(289*ns))

78.89273356401384


## benchmark timing

Here is a comparison of most operations : 

In [1]:
import timeit

operations = {
    "add":"__add__", 
    "sub":"__sub__",
    "mul":"__mul__",
}

In [19]:
import pint
import physipy
import forallpeople
import numpy as np

ureg = pint.UnitRegistry()

a = 123456
b = 654321
arr = np.arange(100)

In [23]:
physipy_qs = {
    "name":"physipy",
    "a":a*physipy.m,
    "b":b*physipy.m,
    'arrm':arr*physipy.m,
}
pint_qs = {
    "name":"pint",
    "a":a*ureg.m,
    "b":b*ureg.m,
    'arrm':arr*ureg.m,
}
fap_qs = {
    "name":"forallpeople",
    "a":a*forallpeople.m,
    "b":b*forallpeople.m, 
    'arrm':arr*forallpeople.m, 
}

for modules_dict in [physipy_qs, pint_qs, fap_qs]:
    print(modules_dict["name"])
    for operation, operation_method in operations.items():
        aq = modules_dict["a"]
        bq = modules_dict["b"]
        #time = timeit.timeit('a.'+operation_method+"(b)", number=10000, globals=globals())
        time_q = timeit.timeit('aq.'+operation_method+"(bq)", number=10000, globals=globals())        
        #print(f"{operation: >5} : {time_q/time: <5.1f}")
        print(f"{operation :>5} : {time_q:.5f}")
    for operation, operation_method in operations.items():
        aq = modules_dict["a"]
        arr = modules_dict["arrm"]
        #time = timeit.timeit('a.'+operation_method+"(b)", number=10000, globals=globals())
        time_qarr = timeit.timeit('aq.'+operation_method+"(arr)", number=10000, globals=globals())
        
        #print(f"{operation: >5} : {time_q/time: <5.1f}")
        print(f"{operation :>5} : {time_qarr:.5f}")
    

physipy
  add : 0.15226
  sub : 0.11834
  mul : 0.22262
  add : 2.31834
  sub : 2.19373
  mul : 2.49435
pint
  add : 0.08753
  sub : 0.08644
  mul : 0.11502
  add : 2.27910
  sub : 2.18198
  mul : 2.37741
forallpeople
  add : 0.02015
  sub : 0.02010
  mul : 0.51084
  add : 4.38325
  sub : 4.35334


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [3]:
pip install line_profiler

     |████████████████████████████████| 89 kB 1.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  ERROR: Command errored out with exit status 1:
   command: /opt/anaconda3/bin/python /opt/anaconda3/lib/python3.8/site-packages/pip/_vendor/pep517/_in_process.py build_wheel /var/folders/5k/bf4syt7x1zjbhc6b28srzzym0000gn/T/tmpaxzjqrf5
       cwd: /private/var/folders/5k/bf4syt7x1zjbhc6b28srzzym0000gn/T/pip-install-osa3ogbz/line-profiler
  Complete output (117 lines):
  Not searching for unused variables given on the command line.
  -- The C compiler identification is unknown
  -- Detecting C compiler ABI info
  -- Detecting C compiler ABI info - failed
  -- Check for working C compiler: /usr/bin/cc
  -- Check for working C compiler: /usr/bin/cc - broken
  CMake Error at /private/var/folders/5k/bf4syt7x1zjbhc6b28srzzym0000gn/T/pip-build-env-y6k2hgd8/overlay/lib/python3.8/site-packages/cmake/data

In [4]:
%lprun

UsageError: Line magic function `%lprun` not found.


# Array creation

Compare lazy creation of arrays

In [ ]:
%timeit asqarray([0*m, 2*m])
%timeit [0, 2]*m

# Profiling base operations

In [7]:
from physipy import m, s, rad, sr

In [ ]:
#%prun -D prunsum -s time m+m 
%prun -D prunsum_file -s nfl m+m 
!snakeviz prunsum_file

 
*** Profile stats marshalled to file 'prunsum_file'. 
snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2FUsers%2Fmocquin%2FDocuments%2FCLE%2FOptique%2FPython%2FJUPYTER%2FMYLIB10%2FMODULES%2Fphysipy%2Fdocs%2Fprunsum_file


Ideas for better performances : 
 - less 'isinstance'
 - remove sympy 
 - cleaner 'setattr'

In [10]:
%%prun -s cumulative -D prundump
m + m
2 * m
2*s /(3*m)
m**3

 
*** Profile stats marshalled to file 'prundump'. 


         453 function calls in 0.000 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 quantity.py:168(__mul__)
       11    0.000    0.000    0.000    0.000 quantity.py:92(__init__)
       66    0.000    0.000    0.000    0.000 quantity.py:100(__setattr__)
       13    0.000    0.000    0.000    0.000 dimension.py:83(__init__)
        5    0.000    0.000    0.000    0.000 quantity.py:1312(quantify)
        3    0.000    0.000    0.000    0.000 dimension.py:143(__mul__)
        1    0.000    0.000    0.000    0.000 quantity.py:182(__truediv__)
        1    0.000    0.000    0.000    0.000 quantity.py:150(__add__)
        1    0.000    0.000    0.000    0.000 quantity.py:213(__pow__)
        5    0.000    0.000    0.000    0.000 decorato

In [ ]:
!snakeviz prundump

snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2FUsers%2Fmocquin%2FDocuments%2FCLE%2FOptique%2FPython%2FJUPYTER%2FMYLIB10%2FMODULES%2Fphysipy%2Fdocs%2Fprundump


# Profiling tests

In [1]:
import sys
sys.path.insert(0,r"/Users/mocquin/MYLIB10/MODULES/physipy/test")
import physipy
import test_dimension
import unittest
from physipy import Quantity, Dimension

In [2]:
from test_dimension import TestClassDimension

In [7]:
%timeit Quantity(1, Dimension(None))

7.4 µs ± 613 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [3]:
%timeit Quantity(1, Dimension(None))

6.72 µs ± 47.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
testdim = test_dimension.TestClassDimension()

In [6]:
testdim.run()

AttributeError: 'TestClassDimension' object has no attribute 'runTest'

In [8]:
%cd ..

/Users/mocquin/Documents/CLE/Optique/Python/JUPYTER/MYLIB10/MODULES/physipy


In [12]:
#%cd
%prun -s module !python -m unittest

..........................................<__array_function__ internals>:5: DeprecationWarning: `np.alen` is deprecated, use `len` instead
....................................................
----------------------------------------------------------------------
Ran 94 tests in 0.524s

OK
 

         8370 function calls in 2.352 seconds

   Ordered by: file name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.352    2.352 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:214(_acquireLock)
        1    0.000    0.000    0.000    0.000 __init__.py:223(_releaseLock)
        1    0.000    0.000    0.000    0.000 _process_posix.py:60(sh)
        1    0.001    0.001    2.250    2.250 _process_posix.py:121(system)
      139    0.001    0.000    0.001    0.000 contextlib.py:82(__init__)
      139    0.000    0.000    0.000    0.000 contextlib.py:108(__enter__)
      139    0.000    0.000    0.001    0.000 contextlib.py:117(__exit__)
      139    0.000    0.000    0.001    0.000 contextlib.py:238(helper)
       40    0.000    0.000    0.000    0.000 exceptions.py:10(__init__)
       42    0.000    0.000    0.000    0.000 expect.py:6(__init__)
      129    0.001    0.000    0.001    0.000 expect.p